## Estimator 的优势
- 用于针对分布式环境训练模型, 例如用大型数据集进行分布式训练并导出模型以用于生产,此外，您可以在 CPU、GPU 或 TPU 上运行基于 Estimator 的模型，而无需重新编码模型。
- Estimator 本身在 tf.layers 之上构建而成，可以简化自定义过程。
- Estimator 会为您构建图。
- Estimator 提供安全的分布式训练循环，可以控制如何以及何时：
  - 构建图
  - 初始化变量
  - 开始排队
  - 处理异常
  - 创建检查点文件并从故障中恢复
  - 保存 TensorBoard 的摘要

使用 Estimator 编写应用时，必须将数据输入管道从模型中分离出来。这种分离简化了不同数据集的实验流程。

## 预创建的 Estimator 程序的结构
1.编写一个或多个数据集导入函数。 例如，您可以创建一个函数来导入训练集，并创建另一个函数来导入测试集。每个数据集导入函数都必须返回两个对象：
- 一个字典，其中键是特征名称，值是包含相应特征数据的张量（或 SparseTensor）
- 一个包含一个或多个标签的张量
```python
def input_fn(dataset):
   ...  # manipulate dataset, extracting the feature dict and the label
   return feature_dict, label
```

2.定义特征列。 每个 tf.feature_column 都标识了特征名称、特征类型和任何输入预处理操作。例如，以下代码段创建了三个存储整数或浮点数据的特征列。前两个特征列仅标识了特征的名称和类型。第三个特征列还指定了一个 lambda，该程序将调用此 lambda 来调节原始数据：
```python
population = tf.feature_column.numeric_column('population')
crime_rate = tf.feature_column.numeric_column('crime_rate')
median_education = tf.feature_column.numeric_column('median_education',
                    normalizer_fn=lambda x: x - global_education_mean)
```

3.实例化相关的预创建的 Estimator
```python
estimator = tf.estimator.LinearClassifier(
    feature_columns=[population, crime_rate, median_education],
    )
```

4.调用训练、评估或推理方法
```python
estimator.train(input_fn=my_training_set, steps=2000)
```

## 自定义 Estimator
每个 Estimator（无论是预创建还是自定义）的核心都是其模型函数，这是一种为训练、评估和预测构建图的方法。
我们推荐以下工作流程：
1. 假设存在合适的预创建的 Estimator，使用它构建第一个模型并使用其结果确定基准。
2. 使用此预创建的 Estimator 构建和测试整体管道，包括数据的完整性和可靠性。
3. 如果存在其他合适的预创建的 Estimator，则运行实验来确定哪个预创建的 Estimator 效果最好。
4. 可以通过构建自定义 Estimator 进一步改进模型。

### 从 Keras 模型创建 Estimator
调用 tf.keras.estimator.model_to_estimator将现有的 Keras 模型转换为 Estimator
```
est_inception_v3 = tf.keras.estimator.model_to_estimator(keras_model=keras_inception_v3)
```

In [2]:
import pandas as pd
import tensorflow as tf

## 1.获取数据

In [1]:
def load_data():
    CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
    y_name = 'Species'

    train_path = './data/iris_training.csv'
    test_path = './data/iris_test.csv'
    
    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)
    return (train_x, train_y), (test_x, test_y)

In [5]:
(train_x, train_y), (test_x, test_y)=load_data()
train_x.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


## 2.创建输入函数
您必须创建输入函数来提供用于训练、评估和预测的数据。

输入函数是 返回tf.data.Dataset对象，此对象接收参数是 含有两个元素的元组：

- features：包含原始输入特征的 {'feature_name':array} 字典（或 DataFrame）。
- labels：包含每个样本的标签的数组。
- batch_size：表示所需批次大小的整数  

```
features = {'SepalLength': np.array([6.4, 5.0]),
            'SepalWidth':  np.array([2.8, 2.3]),
            'PetalLength': np.array([5.6, 3.3]),
            'PetalWidth':  np.array([2.2, 1.0])}
labels = np.array([2, 1])
```

### tf.data.Dataset.from_tensor_slices
 tf.data.Dataset.from_tensor_slices 函数创建一个代表数组切片的 tf.data.Dataset。系统会在第一个维度内对该数组进行切片。例如，一个包含 MNIST 训练数据的数组的形状为 (60000, 28, 28)。将该数组传递给 from_tensor_slices 会返回一个包含 60000 个切片的 Dataset 对象，其中每个切片都是一个 28x28 的图像
```python
mnist_ds = tf.data.Dataset.from_tensor_slices(mnist_x)
```
这段代码将输出以下行，显示数据集中item的形状和类型。请注意，Dataset 不知道自己包含多少item
```
<TensorSliceDataset shapes: (28,28), types: tf.uint8>
```

Dataset 可以透明地处理字典或元组（或 namedtuple）的任何嵌套组合。例如，在将鸢尾花 features 转换为标准 Python 字典后，您可以将数组字典转换为字典 Dataset
```python
dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
```
```
<TensorSliceDataset
    shapes: (
        {
          SepalLength: (), PetalWidth: (),
          PetalLength: (), SepalWidth: ()},
        ()),

    types: (
        {
          SepalLength: tf.float64, PetalWidth: tf.float64,
          PetalLength: tf.float64, SepalWidth: tf.float64},
        tf.int64)>
```
### 操作
Dataset 会按固定顺序迭代数据一次，并且一次仅生成一个元素。它需要进一步处理才可用于训练
```python
dataset = dataset.shuffle(1000).repeat().batch(batch_size)
```
- shuffle 方法使用一个固定大小的缓冲区，在条目经过时随机化处理条目。buffer_size 大于 Dataset 中样本的数量，确保数据完全被随机化处理（鸢尾花数据集仅包含 150 个样本）。

- repeat 方法会在结束时重启 Dataset。要限制周期数量，请设置 count 参数。

- batch 方法会收集大量样本并将它们堆叠起来以创建批次。这为批次的形状增加了一个维度。新的维度将添加为**第一个维度**。以下代码对之前的 MNIST Dataset 使用 batch 方法。
```
print(mnist_ds.batch(100))
```
```
<BatchDataset
  shapes: (?, 28, 28),
  types: tf.uint8>
```
请注意，该数据集的批次大小是未知的，因为最后一个批次具有的元素数量会减少。

在 train_input_fn 中，经过批处理之后，dataset如下所示：
```
print(dataset)
```
```
<TensorSliceDataset
    shapes: (
        {
          SepalLength: (?,), PetalWidth: (?,),
          PetalLength: (?,), SepalWidth: (?,)},
        (?,)),

    types: (
        {
          SepalLength: tf.float64, PetalWidth: tf.float64,
          PetalLength: tf.float64, SepalWidth: tf.float64},
        tf.int64)>
```
### 返回
此时，Dataset 包含 (features_dict, labels) 对。这是 train 和 evaluate 方法的预期格式，因此 input_fn 会返回相应的数据集

使用 predict 方法时，可以/应该忽略 labels

In [6]:
def train_input_fn(features, labels, batch_size):

    # 将输入转化为Dataset对象
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    return dataset


def eval_input_fn(features, labels, batch_size):
    features = dict(features)
    if labels is None:
        inputs = features  # No labels, use only features.
    else:
        inputs = (features, labels)

    # 将inputs转化为Dataset对象
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # batch the examples
    dataset = dataset.batch(batch_size)

    return dataset

## 3.定义特征列
特征列是一个对象，用于说明模型应该如何使用特征字典中的原始输入数据。在构建 Estimator 模型时，向其传递一个特征列的列表，其中包含您希望模型使用的每个特征。tf.feature_column 模块提供很多用于在模型中表示数据的选项。
![](images/1.jpg)
![](images/2.jpg)
对于鸢尾花问题，4 个原始特征是数值，因此我们会构建一个特征列的列表，以告知 Estimator 模型将这 4 个特征都表示为 32 位浮点值。因此，创建特征列的代码如下所示：

In [7]:
my_feature_columns = []
for key in train_x.keys():
    print(key)
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))  # 默认dtype=tf.float32
    print(my_feature_columns)

SepalLength
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
SepalWidth
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
PetalLength
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]
PetalWidth
[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key

## 4.实例化 Estimator
鸢尾花问题是一个经典的分类问题。幸运的是，TensorFlow 提供了几个预创建的分类器 Estimator，其中包括：

tf.estimator.DNNClassifier：适用于执行多类别分类的深度模型。  
tf.estimator.DNNLinearCombinedClassifier：适用于宽度和深度模型。  
tf.estimator.LinearClassifier：适用于基于线性模型的分类器。  

对于鸢尾花问题，tf.estimator.DNNClassifier 似乎是最好的选择
```
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3
    model_dir='models/iris'  # 检查点文件目录
    )  
```
### 默认检查点目录
第一次调用 train 会将检查点和其他文件添加到 model_dir 目录中  
如果未在 Estimator 的构造函数中指定 model_dir，则 Estimator 会将检查点文件写入由 Python 的 tempfile.mkdtemp 函数选择的临时目录中
```python
print(classifier.model_dir)
```

### 检查点频率  
**默认情况**下，Estimator 按照以下时间安排将检查点保存到 model_dir 中：  

- 每 10 分钟（600 秒）写入一个检查点。  
- 在 train 方法开始（第一次迭代）和完成（最后一次迭代）时写入一个检查点。  
- 只在目录中保留 5 个最近写入的检查点。  

您可以通过执行下列步骤来更改默认时间安排：  

- 创建一个 RunConfig 对象来定义所需的时间安排。  
- 在实例化 Estimator 时，将该 RunConfig 对象传递给 Estimator 的 config 参数。  

```python
my_checkpointing_config = tf.estimator.RunConfig(
        save_checkpoints_secs = 20*60,  # Save checkpoints every 20 minutes.
        keep_checkpoint_max = 3,       # 只保留最近3个checkpoints.
    )

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[10, 10],
    n_classes=3,
    model_dir='models/iris',
    config=my_checkpointing_config)
```

### 恢复模型

第一次调用 Estimator 的 train 方法时，TensorFlow 会将一个检查点保存到 model_dir 中。随后每次调用 Estimator 的 train、evaluate 或 predict 方法时，都会发生下列情况：
- Estimator 通过运行 model_fn() 构建模型图。
- Estimator 根据最近写入的检查点中存储的数据来初始化新模型的权重。  

换言之，一旦存在检查点，TensorFlow 就会在您每次调用 train()、evaluate() 或 predict() 时重建模型。

### 避免不当恢复
通过检查点恢复模型的状态这一操作仅在模型和检查点兼容时可行。例如，假设您训练了一个 DNNClassifier Estimator，它包含 2 个隐藏层且每层都有 10 个节点：  

在训练之后（因此已在 models/iris 中创建检查点），假设您将每个隐藏层中的神经元数量从 10 更改为 20，然后尝试重新训练模型：

由于检查点中的状态与 classifier2 中描述的模型不兼容，因此重新训练失败并出现错误

**可以在重新训练前删除检查点文件**

## 5.训练、评估和预测
- 通过调用 Estimator 的 train 方法训练模型，如下所示:
```python
classifier.train(
    input_fn=lambda: train_input_fn(train_x, train_y, batch_size),
    steps=train_steps)
```
将 input_fn 调用封装在 lambda 中以获取参数，同时提供一个不采用任何参数的输入函数。steps 参数告知方法在训练多步后停止训练

- 调用 Estimator 的 evaluate 方法评估经过训练的模型
```python
eval_result = classifier.evaluate(
    input_fn=lambda: eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
```
eval_result的结果为：
{'accuracy': 0.96666664, 'average_loss': 0.05575007, 'loss': 1.672502, 'global_step': 2000}

与对 train 方法的调用不同，我们没有传递 steps 参数来进行评估。我们的 eval_input_fn 只生成一个epoch的数据

- 调用 Estimator 的 predict 方法进行预测

```python
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda: eval_input_fn(predict_x, batch_size=batch_size))
for pred_dict, expec in zip(predictions, expected):
```
train_x,test_x,predict_x 都要转化为python字典  
predictions为：  
```
<generator object Estimator.predict at 0x7f2d514c1e08>
```
1个pred_dict为1个样本的预测结果：
```
{'logits': array([ 20.65349 ,  15.137624, -16.802929], dtype=float32), 'probabilities': array([9.9599361e-01, 4.0063257e-03, 5.3844290e-17], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
```

## 完整代码

In [2]:
import pandas as pd
import tensorflow as tf

batch_size=100
train_steps=1000
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def load_data():
    CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
    y_name = 'Species'

    train_path = './data/iris_training.csv'
    test_path = './data/iris_test.csv'
    
    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)
    return (train_x, train_y), (test_x, test_y)


def train_input_fn(features, labels, batch_size):

    # 将输入转化为Dataset对象
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    return dataset


def eval_input_fn(features, labels, batch_size):
    features = dict(features)
    if labels is None:
        inputs = features  # No labels, use only features.
    else:
        inputs = (features, labels)

    # 将inputs转化为Dataset对象
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # batch the examples
    dataset = dataset.batch(batch_size)

    return dataset


def main():
    # 获取数据
    (train_x, train_y), (test_x, test_y) = load_data()

    # 定义特征列
    # key是train_x的列名：SepalLength，SepalWidth，PetalLength，PetalWidth
    my_feature_columns = []
    for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    my_checkpointing_config = tf.estimator.RunConfig(
        save_checkpoints_secs = 20*60,  # Save checkpoints every 20 minutes.
        keep_checkpoint_max = 3,       # 只保留最近3个checkpoints.
    )

    # 建立2个隐藏层DNN的模型，每层10个隐藏节点
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,  #特征列
        hidden_units=[10, 10],  #每层10个节点
        n_classes=3,  #分为3类
        model_dir='models/iris',  # 保存模型
        config=my_checkpointing_config
    )  

    # 训练模型
    classifier.train(
        input_fn=lambda: train_input_fn(train_x, train_y, batch_size),
        steps=train_steps)

    # 评估模型
    eval_result = classifier.evaluate(
        input_fn=lambda: eval_input_fn(test_x, test_y, batch_size))
    
    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    # 进行预测
    # predict_x里的1列为1个样本，有3列共3个样本，expected是3个样本的真实label
    expected = ['Setosa', 'Versicolor', 'Virginica']
    predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth':  [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth':  [0.5, 1.5, 2.1],
    }

    predictions = classifier.predict(
        input_fn=lambda: eval_input_fn(predict_x, labels=None, batch_size=batch_size))

    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    for pred_dict, expec in zip(predictions, expected):
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(SPECIES[class_id], 100 * probability, expec))


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.ERROR)
    main()



Test set accuracy: 0.967


Prediction is "Setosa" (99.7%), expected "Setosa"

Prediction is "Versicolor" (100.0%), expected "Versicolor"

Prediction is "Virginica" (99.6%), expected "Virginica"


# 创建自定义 Estimator
预创建的 Estimator
![](images/3.png)
如果需要设计独特的模型或指标，也可以编写自定义 Estimator  

以下使用自定义 Estimator 解决鸢尾花问题

## 自定义模型函数
自定义模型函数参考下面标准:
```python
def my_model_fn(
   features, # This is batch_features from input_fn
   labels,   # This is batch_labels from input_fn
   mode,     # An instance of tf.estimator.ModeKeys
   params):  # Additional configuration
```

前两个参数是从输入函数中返回的特征和标签批次；也就是说，features 和 labels 是模型将使用的数据的句柄。  
mode 参数表示调用程序是请求训练、预测还是评估   
调用程序可以将 params 传递给 Estimator 的构造函数。传递给构造函数的所有 params 转而又传递给 model_fn。
### 定义模型
必须定义下列三个部分：
- 一个输入层
- 一个或多个隐藏层
- 一个输出层

**输入层**  
在 model_fn 的第一行调用 tf.feature_column.input_layer，以将特征字典和 feature_columns 转换为模型的输入，如下所示：
```python
net = tf.feature_column.input_layer(features, params['feature_columns'])
```
**隐藏层**  
使用tf.layers  
在第一次迭代中，net 表示输入层。在每次循环迭代时，tf.layers.dense 都使用变量 net 创建一个新层，该层将前一层的输出作为其输入。
```python
for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)  #units 参数会定义指定层中输出神经元的数量。
```
创建两个隐藏层后，我们的网络如下所示:
![](images/5.png)
**输出层**  
输出层不使用激活函数
```python
logits = tf.layers.dense(net, params['n_classes'], activation=None)
```
![](images/6.png)
### 实现训练、评估和预测
Estimator方法 | Estimator模式
-|-
train() | ModeKeys.TRAIN
evaluate() | ModeKeys.EVAL
predict() | ModeKeys.PREDICT

**预测**   
如果调用 Estimator 的 predict 方法，则 model_fn 会收到 mode = ModeKeys.PREDICT。在这种情况下，模型函数必须返回一个包含预测的 tf.estimator.EstimatorSpec
该模型必须经过训练才能进行预测
```python
predicted_classes = tf.argmax(logits, 1)
if mode == tf.estimator.ModeKeys.PREDICT:
    # 定义predictions输出哪些内容
    predictions = {
        'class_ids': predicted_classes[:, tf.newaxis],
        'probabilities': tf.nn.softmax(logits),
        'logits': logits,
    }
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)
```
针对预测返回的 EstimatorSpec 通常包含 predictions 参数,用于将该字典返回到调用程序。Estimator 的 predict 方法会生成这些字典

**计算损失**  
通过调用 tf.losses.sparse_softmax_cross_entropy 计算交叉熵损失  
此函数会针对整个批次返回平均值
```python
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
```
**评估**  
如果调用 Estimator 的 evaluate 方法，则 model_fn 会收到 mode = ModeKeys.EVAL。在这种情况下，模型函数必须返回一个包含模型损失和一个或多个指标（可选）的 tf.estimator.EstimatorSpec  
tf.metrics 来计算常用指标,tf.metrics.accuracy 函数要求标签和预测具有相同的形状
```python
accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op')
```

针对评估返回的 EstimatorSpec 通常包含:
- loss：这是模型的损失
- eval_metric_ops：这是可选的指标字典

```python
metrics = {'accuracy': accuracy}
tf.summary.scalar('accuracy', accuracy[1])  #tf.summary.scalar 会在 TRAIN 和 EVAL 模式下向 TensorBoard 提供准确率

if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(
        mode, loss=loss, eval_metric_ops=metrics)
```
**训练**  
如果调用 Estimator 的 train 方法，则会调用 model_fn 并收到 mode = ModeKeys.TRAIN。在这种情况下，模型函数必须返回一个包含损失和训练操作的 EstimatorSpec  
需要构建训练操作需要优化器和train_op  
针对训练返回的 EstimatorSpec 必须包含:
- loss：包含损失函数的值。
- train_op：执行训练步

## 实例化自定义Estimator
通过 Estimator 基类实例化自定义 Estimator
```python
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'feature_columns': my_feature_columns,
        'hidden_units': [10, 10],
        'n_classes': 3,
    })
```


## 完整代码

In [7]:
import pandas as pd
import tensorflow as tf

batch_size=100
train_steps=1000
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def load_data():
    CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
    y_name = 'Species'

    train_path = './data/iris_training.csv'
    test_path = './data/iris_test.csv'
    
    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)
    return (train_x, train_y), (test_x, test_y)


def train_input_fn(features, labels, batch_size):
    # 将输入转化为Dataset对象
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset


def eval_input_fn(features, labels, batch_size):
    features = dict(features)
    if labels is None:
        inputs = features  # No labels, use only features.
    else:
        inputs = (features, labels)

    # 将inputs转化为Dataset对象
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # batch the examples
    dataset = dataset.batch(batch_size)

    return dataset


# 自定义模型,重写4个参数
def my_model(features, labels, mode, params):
    # 定义输入层
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    # 定义隐藏层
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # 定义输出层
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # 计算predictions,返回tf.estimator.EstimatorSpec
    predicted_classes = tf.argmax(logits, 1)
    
    # 模型预测
    if mode == tf.estimator.ModeKeys.PREDICT:
        # 定义predictions输出哪些内容
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # 计算loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # 计算evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])
    
    # 评估模型
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)



def main():
    # 获取数据
    (train_x, train_y), (test_x, test_y) = load_data()

    # 定义特征列
    # key是train_x的列名：SepalLength，SepalWidth，PetalLength，PetalWidth
    my_feature_columns = []
    for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    my_checkpointing_config = tf.estimator.RunConfig(
        save_checkpoints_steps = 100,  # 每100步保存，不能和save_checkpoints_secs同时设置
        keep_checkpoint_max = 3,       # 只保留最近3个checkpoints.
    )
    
    # 建立2个隐藏层DNN的模型，每层10个隐藏节点
    classifier = tf.estimator.Estimator(
        model_fn=my_model,
        params={
            'feature_columns': my_feature_columns,
            'hidden_units': [10, 10],
            'n_classes': 3,
        },
        model_dir='models/iris',  # 保存模型
        config=my_checkpointing_config
    )

    # 训练模型
    classifier.train(
        input_fn=lambda: train_input_fn(train_x, train_y, batch_size),
        steps=train_steps)

    # 评估模型
    eval_result = classifier.evaluate(
        input_fn=lambda: eval_input_fn(test_x, test_y, batch_size))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    # 进行预测
    # predict_x里的1列为1个样本，有3列共3个样本，expected是3个样本的真实label
    expected = ['Setosa', 'Versicolor', 'Virginica']
    predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }

    predictions = classifier.predict(
        input_fn=lambda: eval_input_fn(predict_x, labels=None, batch_size=batch_size))

    for pred_dict, expec in zip(predictions, expected):
        template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(SPECIES[class_id], 100 * probability, expec))


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.ERROR)
    main()


Test set accuracy: 0.967


Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (99.9%), expected "Versicolor"

Prediction is "Virginica" (95.8%), expected "Virginica"
